# Sparse Recovery Algorithms

This Jupyter notebook provides implementations of the sparse recovery algorithms Orthogonal Matching Pursuit (OMP),
Compressive Sampling Matching Pursuit (CoSaMP) and Iterative Hard Thresholding (IHT). We take $m$ random measurements of an $s$-sparse vector $x \in \mathbb{R}^n$ and want to recover $x$ from its measurement $y \in \mathbb{R}^m$. We consider the standard case of a Gaussian measurement matrix $A \in \mathbb{R}^{m \times n}$ where
$A_{i_j} \sim \mathcal{N}(0,1/m)$ i.i.d., i.e. the measurement takes the form $A x = y$ and we wish to recover $x$ from only $A$ and $y$ given, with ideally as few measurements $m$ as possible (i.e. each row of $A$ corresponds to one random measurement). For an elementary introduction to Compressive Sensing, and pseudocode and references to the algorithms considered below, see the following survey. 

http://people.fjfi.cvut.cz/vybirja2/publ/Chapter1.pdf

## Generate the Data and the Hard Thresholding Operator




In [2]:
import numpy as np
import random

The following function generates an $s$-sparse $n$-dimensional vector $x$ by first choosing the support from a uniform distribution, and then assigning normally distributed random numbers to the support (which may be modified), and samples a measurement matrix $A \in \mathbb{R}^{m \times n}$ according to the distribution above, where $m$ is the number of measurements.

In [3]:
def generate_sparse_data_and_measurement(n,m,s):
    x = np.zeros((n,1))
    x[0:s,:] = 10*np.random.randn(s,1)
    np.random.shuffle(x)
    A = np.random.randn(m,n) / np.sqrt(m)
    y = A @ x
    
    return A,x,y  

In [4]:
A,x,y = generate_sparse_data_and_measurement(n=100,m=50,s=5)

# Print x if you wish
print(x)

[[  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [-19.13221952]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.  

Defines the hard thresholding operator $H_k$ on some input vector $x$. The function returns both the indecies of the $k$-largest entries as a list (as the first argument), as well as the setting the entries that are not in the list to zero, i.e. returning the best $k$-sparse approximation of $x$ as a column vector/array (as the second argument).


In [5]:
def hard_thresholding(x,k):
    x = x.flatten()
    length = x.size
    H_k_x = np.zeros_like(x)
    indices = list(np.argsort(np.absolute(x))[::-1][0:k].flatten())
    H_k_x[indices] = x[indices]
    H_k_x = np.reshape(H_k_x, (length, 1))  

    return indices, H_k_x

In [6]:
### Simple Example:
#a = np.array([[-10,2,-4, 3,4,-5]])
#hard_thresholding(a,k=2)

### Orthogonal Matching Pursuit (OMP)

In [7]:
def omp(A,x):
    
    s = np.count_nonzero(x)
    x_hat = np.zeros_like(x)
    y = A @ x
    r = np.copy(y)
    T = []
    
    for i in range(s):
        index_largest_entry = np.argmax(np.abs(A.T @ r))
        T.append(index_largest_entry)
        x_hat[T,:] = np.linalg.pinv(A[:,T]) @ y
        r = y - A @ x_hat
        
    return x_hat

omp_recovery = omp(A,x)
print(omp_recovery)
print('reconstruction error is', np.linalg.norm(x-omp_recovery))

[[  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [-19.13221952]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.  

### Compressive Sampling Matching Pursuit (CoSaMP) 

In [8]:
def cosamp(A,x):

    s = np.count_nonzero(x)
    x_hat = np.zeros_like(x)
    y = A @ x
    r = np.copy(y)
    T = []
    A_pinv_T_i = np.zeros_like(A)
    
    for i in range(s):
        indices = hard_thresholding(A.T @ r,2*s)[0]
        supp_x_hat = x_hat.flatten().nonzero()[0].tolist()
        T = list(set().union(T,indices,supp_x_hat))
        A_pinv_T_i[:,T] = A[:,T]
        xhat = hard_thresholding(np.linalg.pinv(A_pinv_T_i) @ y, s)[1]
        r = y - A @ xhat
        
    return xhat

cosamp_recovery = cosamp(A,x)
print(cosamp_recovery)
print(np.linalg.norm(x-cosamp_recovery))

[[  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [-19.13221952]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.  

### Iterative Hard Thresholding (IHT)

In [9]:
def iht(A,x):
    
    s = np.count_nonzero(x)
    x_hat = np.zeros_like(x)
    y = A @ x
    
    for i in range(s):
        x_hat = hard_thresholding(x_hat + A.T @ (y - A @ x_hat),s)[1]
        
    return x_hat

iht_recovery = omp(A,x)
print(iht_recovery)
print('reconstruction error is', np.linalg.norm(x-iht_recovery))

[[  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [-19.13221952]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [  0.  